<a href="https://colab.research.google.com/github/ronaldobonilla/Lab6-HAR-with-LSTM/blob/master/Lab6_HAR_with_LSTM_16001316.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPORT TENSORFLOW

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only works in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


LSTM Model for HAR dataset



In [0]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot

In [0]:
dataset_path = "/content/drive/My Drive/Lab6-SP1/"

In [0]:
# run this only once to save dataset to drive
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/HAR_Smartphones.zip -P "/content/drive/My Drive/Lab6-SP1/"

--2020-03-08 20:03:05--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/HAR_Smartphones.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60999314 (58M) [application/zip]
Saving to: ‘/content/drive/My Drive/Lab6-SP1/HAR_Smartphones.zip.1’

HAR_Smartphones.zip 100%[===================>]  58.17M  57.4MB/s    in 1.0s    

2020-03-08 20:03:07 (57.4 MB/s) - ‘/content/drive/My Drive/Lab6-SP1/HAR_Smartphones.zip.1’ saved [60999314/60999314]



In [0]:
!unzip -q "/content/drive/My Drive/Lab6-SP1/HAR_Smartphones.zip" -d "/content/drive/My Drive/Lab6-SP1/"

replace /content/drive/My Drive/Lab6-SP1/UCI HAR Dataset/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/Lab6-SP1/__MACOSX/UCI HAR Dataset/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/Lab6-SP1/UCI HAR Dataset/activity_labels.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/Lab6-SP1/__MACOSX/UCI HAR Dataset/._activity_labels.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/Lab6-SP1/UCI HAR Dataset/features.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/Lab6-SP1/__MACOSX/UCI HAR Dataset/._features.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/Lab6-SP1/UCI HAR Dataset/features_info.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace /content/drive/My Drive/Lab6-SP1/UCI HAR Dataset/features_info.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [0]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

In [0]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

In [0]:
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

In [0]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + '/content/drive/My Drive/Lab6-SP1/UCI HAR Dataset/')
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_group('test', prefix + '/content/drive/My Drive/Lab6-SP1/UCI HAR Dataset/')
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy

In [0]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 0, 15, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

In [0]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [0]:
# run an experiment
def run_experiment(repeats=10):
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

In [53]:
# run the experiment
run_experiment()


(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 92.026
>#2: 91.653
>#3: 90.261
>#4: 89.650
>#5: 88.565
>#6: 89.786
>#7: 89.515
>#8: 89.888
>#9: 91.483
>#10: 90.261
[92.02578663825989, 91.65253043174744, 90.26128053665161, 89.65049386024475, 88.5646402835846, 89.78622555732727, 89.51476216316223, 89.88802433013916, 91.48286581039429, 90.26128053665161]
Accuracy: 90.309% (+/-1.034)
